**Hugging Face Transformers 微调训练入门**

In [17]:
from datasets import Dataset, DatasetDict

In [18]:
# Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。
train_full_ds = Dataset.from_parquet("r_data/yelp_review_full/train-00000-of-00001.parquet")
test_full_ds = Dataset.from_parquet("r_data/yelp_review_full/test-00000-of-00001.parquet")

In [19]:
dataset = DatasetDict({
    'train': train_full_ds,
    'test': test_full_ds
})

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [21]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [22]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [23]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [24]:
show_random_elements(dataset["train"])

,label,text
0,3 stars,Love this club because: \n\n- It's intimate\n- Easy to mingle\n- Fun crowd! \n- Can never turn down unlimited champagne before midnight...or was it\n 1pm?? Regardless.\n- Decor\n\nThere! This must be my shortest review.
1,2 star,"Maybe I am just getting pickier with each review and less generous with my stars, but I am trying to figure out why the previous reviewers raved so much about Culinary Dropout. There are certainly things to love . . . the expansive outdoor patio with some of the most comfortable seating around. It's great for people-watching as it fronts Camelback Road, just across the street from Scottsdale Fashion Square. On a hot and hazy summer afternoon, we were still very content sitting on the patio.\n\nOn the negative side . . . The food was not even so-so. I expected so much more from a Sam Fax Restaurant. We have been fans for years with True Food, North, The Greenhouse and the now-closed Bloom, and they are on our regular dining rotation. Culinary Dropout was how its name sounded, someone dropped the culinary part when they made our meals. I had a hard time finding something that even sounded appetizing on the menu! We started with the potato chips and onion dip. Nothing to write home about, although I think that if I had a real salt craving, these might just satisfy it. Maybe I should have ordered the pretzel and provolone fondue . . . \n\nFor an entree I ordered the ricotta gnocchi with sausage and mushroom . . . sure sounded like a winner . . . NOT! The gnocchi and sauce were pretty tasteless, and the mushrooms and sausage were so salty I couldn't eat them. Mix them all together and it tasted even worse. My husband ordered the Beef Stroganoff and was very disappointed. There was no sour cream taste whatsoever. Boring was what he called it.\n\nSo the food is not up to par with the rest of the Fox restaurants, but the worst part was the service. The place was pretty empty, and while our server was attentive to our needs prior to our order, after our food was delivered, he pretty much disappeared -- no water refills, no iced tea refills . . . nothing. On a 106 degree day, that should never happen on the patio!!\n\nNeedless to say, we won't be making any trips back to Culinary Dropout!"
2,1 star,"SKIP THE DOGS! They're HORRIBLE! I mean, TERRIBLE! The chili was definitely delicious, so if you're really stuck on going here, grab the chili cheese fries and please save yourself the money and skip the dog. I've never been so adamant on telling someone to skip something as I am about this! The bun was stale, the dog itself was bad- it had that POP when you take your first bite, and though I know that is the goal of some places that serve dogs, I know that's not the goal for pinks and I'm pretty sure it poped because it was way overly old and over cooked. I'm sad to have to give a place with such a great reputation in LA such a bad review, but clearly they sold themselves out when coming to Vegas and just got dollar signs in their eyes without making sure the quality followed with them. I seriously have had MUCH BETTER hot dogs at the movie theater. I couldn't even finish my over-priced hot dog because it was THAT BAD. I know I sound like a snob, but I'm not, i promise. I was hungry after having a long day of walking and beers and all I wanted was a delicious juicy hot dog and this was very disappointing. I keep going on but it was that bad. Good luck to you if you ignore this and still get a dog, I pray I just caught them on a really really off and bad day, but from what I understand, I'm not too far off from what others were saying.Sorry friends!"
3,2 star,"Came here because of the buffet PASS. I should've PASSed on this one!\n\nThis was almost as bad as the Imperial Palace buffet!!! Bad customer service, it took 20 mins for the person to verify our room and pass. It took the host 10 minutes to seat us. (mind you, we were the only one in line at this time). The selections were ridiculously f

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [25]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/data/models/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [26]:
tokenized_datasets.save_to_disk("r_data/yelp_review_full/tokenized_datasets")

Saving the dataset (0/5 shards):   0%|          | 0/650000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [27]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [1]:
from datasets import load_from_disk
tokenized_datasets = load_from_disk("r_data/yelp_review_full/tokenized_datasets")

### 数据抽样

使用 N 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [2]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=66).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=66).select(range(100))

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/data/models/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /data/models/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：
- https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments
- https://hf-mirror.com/docs/transformers/training

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [4]:
from transformers import TrainingArguments

model_dir = "/data/models/bert-base-cased-finetune-yelp-100"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为50
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=50)

In [5]:
TrainingArguments?

Init signature:
TrainingArguments(
    output_dir: Optional[str] = None,
    overwrite_output_dir: bool = False,
    do_train: bool = False,
    do_eval: bool = False,
    do_predict: bool = False,
    eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no',
    prediction_loss_only: bool = False,
    per_device_train_batch_size: int = 8,
    per_device_eval_batch_size: int = 8,
    per_gpu_train_batch_size: Optional[int] = None,
    per_gpu_eval_batch_size: Optional[int] = None,
    gradient_accumulation_steps: int = 1,
    eval_accumulation_steps: Optional[int] = None,
    eval_delay: Optional[float] = 0,
    torch_empty_cache_steps: Optional[int] = None,
    learning_rate: float = 5e-05,
    weight_decay: float = 0.0,
    adam_beta1: float = 0.9,
    adam_beta2: float = 0.999,
    adam_epsilon: float = 1e-08,
    max_grad_norm: float = 1.0,
    num_train_epochs: float = 3.0,
    max_steps: int = -1,
    lr_scheduler_type: Union[transformers.trainer_utils.Schedu

In [6]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.NO,
eval_use_gather_object=False,


### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [7]:
import numpy as np
import evaluate

In [ ]:
# metric = evaluate.load("accuracy")

接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [ ]:
# def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  # evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

evaluation_strategy="epoch" 可以看每次的评估结果，可以判断是否过拟合的情况。但是我这里一致报错，不知道问题原因，就注释掉了。

## 开始训练

### 实例化训练器（Trainer）

需要升级accelerate
```bash
pip uninstall accelerate -y
pip cache purge
pip install accelerate --upgrade
```

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [9]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=21, training_loss=1.5325753348214286, metrics={'train_runtime': 2088.3473, 'train_samples_per_second': 0.144, 'train_steps_per_second': 0.01, 'total_flos': 78935442739200.0, 'train_loss': 1.5325753348214286, 'epoch': 3.0})

In [10]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(50))

In [11]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.6635388135910034,
 'eval_accuracy': 0.22,
 'eval_runtime': 133.4795,
 'eval_samples_per_second': 0.375,
 'eval_steps_per_second': 0.052,
 'epoch': 3.0}

使用1000个样本训练，使用100个随机样本测试，结果如下
```
{'eval_loss': 1.0753791332244873,
 'eval_accuracy': 0.52,
 'eval_runtime': 2.9889,
 'eval_samples_per_second': 33.457,
 'eval_steps_per_second': 4.349,
 'epoch': 3.0}
```
 说明样本量越大，精度会同步提升。

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [12]:
trainer.save_model(model_dir)

In [13]:
trainer.save_state()